In [ ]:
!pip install --upgrade openai --user
!pip install openai 

In [ ]:
import pandas as pd
df = pd.read_csv('')
df.head()

,Unnamed: 0.1,Unnamed: 0,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,Accumulated_Utterance,history_prediction,image_caption,audio_caption,Accumulated_Utterance_by_prediction,three_accum_utt
0,0,0,1,Why do all you're coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",Mark : Why do all you're coffee mugs have numb...,0,A woman and a man standing in a kitchen.,An adult male is speaking,Mark : Why do all you're coffee mugs have numb...,Mark : Why do all you're coffee mugs have numb...
1,1,1,2,Oh. That's so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",Mark : Why do all you're coffee mugs have numb...,1,A woman standing next to a man in a kitchen.,"An adult female is speaking, and a crowd laughs",Rachel : Oh. That's so Monica can keep track. ...,Mark : Why do all you're coffee mugs have numb...
2,2,2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",Mark : Why do all you're coffee mugs have numb...,1,A black cat is sitting on a table.,A woman and a man are speaking to each other.,Rachel : Oh. That's so Monica can keep track. ...,Mark : Why do all you're coffee mugs have numb...
3,3,3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146","Joey : Come on, Lydia, you can do it.",0,A woman and a baby are in a hospital bed.,People are talking and laughing,"Joey : Come on, Lydia, you can do it.","Joey : Come on, Lydia, you can do it."
4,4,4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833","Joey : Come on, Lydia, you can do it. $$$ Joey...",0,A woman and a child are in a hospital bed.,A man is speaking and a cat is meowing,"Joey : Come on, Lydia, you can do it. $$$ Joey...","Joey : Come on, Lydia, you can do it. $$$ Joey..."


In [ ]:
#for history(prev3)
def add_three_accum_utt(df):
    df['three_accum_utt'] = ''

    for dialogue_id, group in df.groupby('Dialogue_ID'):
        for idx, row in group.iterrows():
            utt_id = row['Utterance_ID']
            speaker = row['Speaker']

            if utt_id == 0:
                df.at[idx, 'three_accum_utt'] = f"{speaker} : {row['Utterance']}"
            elif utt_id == 1:
                accum_utt = []
                for i in range(utt_id):
                    prev_row = group[group['Utterance_ID'] == i].iloc[0]
                    accum_utt.append(f"{prev_row['Speaker']} : {prev_row['Utterance']}")
                accum_utt.append(f"{speaker} : {row['Utterance']}")
                df.at[idx, 'three_accum_utt'] = ' $$$ '.join(accum_utt)
            else:
                accum_utt = []
                for i in range(max(0, utt_id - 2), utt_id + 1):
                    if i in group['Utterance_ID'].values:
                        prev_row = group[group['Utterance_ID'] == i].iloc[0]
                        accum_utt.append(f"{prev_row['Speaker']} : {prev_row['Utterance']}")
                df.at[idx, 'three_accum_utt'] = ' $$$ '.join(accum_utt)

    return df

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# history + captionings
for index, row in df.iterrows():
    completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": '''You are a psychologist with great ability to predict the feelings of the last utterance of the following drama conversation. 

    1. I will give you the drama dialouge and 2 helpful situation information. In other words, I will let you know 3 things. 
    a) The dialouge (utterances)
    b) Description of the visual information (frame) of the drama
    c) Description of the audio information (audio) of the drama

    2. Using this information, you must judge the emotion of the last utterance of the conversation.

    3. In the conversation, each utterance is separated by $$$. You just have to predict the emotion of the last utterance.
    For example, {Speaker's name1}: {Utterance1}$$${Speaker's name2}:{Utterance2}$$${Speaker's name3}:{Utterance3}. => You have to predict the emotion of utterance3.


    4. The last utterance is speaking with one of the following seven emotions. Never predict another emotion except for these seven sentiments. 
    <neutral, anger, surprise, joy, disgust, sadness, fear>
    You must predict the last utterspeaker's emotion among seven emotions.

    5. Your output format should be next. 
    <emotion : {your_prediction}>

    You can't talk about the basis of judgment, the situation, only to keep that form all the time.

    For example, if you judged that the emotion of the last utterance was a specific sentiment1, your output should be <emotion: {sentiment1}>.'''},
    {"role": "user", "content": f"(a) the dialogue : {row['Accumulated_Utterance_by_prediction']} \n (b) Description of the visual information : {row['image_caption']} \n (c) Description of the audio information :  {row['audio_caption']}"}  
  ]
)
    assistant_response = completion.choices[0].message.content
    print(assistant_response)
    df.at[index, 'prediction'] = assistant_response
    df.to_csv(r"C:\Users\Yoon\OneDrive\한양대학교\3-1\인공지능\pj\pj\ssakda_GPT_4o_History_final_output.csv")


<emotion : neutral>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: joy>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: joy>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: irritation>
<emotion: frustration>
<emotion: neutral>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: neutral>
<emotion: frustration>
<emotion: neutral>
<emotion: neutral>
<emotion: surprise>
<emotion: frustration>
<emotion: neutral>
<emotion: disgust>
<emotion: joy>
<emotion: surprise>
<emotion: surprise>
<emotion: surprise>
<emotion: joy>
<emotion: disgust>
<emotion: fear>
<emotion: disgust>
<emotion: fear>
<emotion: fear>
<emotion: surprise>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: joy>
<emotion: joy>
<emotion: neutral>
<emotion: neutral>
<emotion: surprise>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: sur

In [ ]:
# Prompt simple & alter + Only Utterance
for index, row in df.iterrows():
    completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": ''' 
     You are the greatest psychologist with ability to predict the feeling of utterance. 
     Predict the emotion of this utterance among <neutral, anger, surprise, joy, disgust, sadness, fear>. 
     You must predict one emotion among seven emotions.
     your output format should be next. 
     <emotion : {your_prediction}>
'''},
    {"role": "user", "content": f"{row['Utterance']})"}  
  ]
)
    assistant_response = completion.choices[0].message.content
    print(assistant_response)
    df.at[index, 'prediction'] = assistant_response
    df.to_csv(r"C:\Users\Yoon\OneDrive\한양대학교\3-1\인공지능\pj\pj\al_gandan_GPT_4o_History_final_output.csv")


<emotion : surprise>
<emotion: surprise>
<emotion : surprise>
<emotion: joy>
<emotion : surprise>
<emotion : anger>
<emotion: joy>
<emotion : joy>
<emotion: surprise>
<emotion : anger>
<emotion: anger>
<emotion : neutral>
<emotion: surprise>
<emotion: neutral>
<emotion: anger>
<emotion : surprise>
<emotion: neutral>
<emotion : surprise>
<emotion: joy>
<emotion : joy>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion : fear>
<emotion : neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: anger>
<emotion : joy>
<emotion : surprise>
<emotion : surprise>
<emotion : joy>
<emotion: surprise>
<fear : fear>
<emotion : surprise>
<emotion: disgust>
<emotion : frustration>
<emotion: joy>
<emotion : surprise>
<emotion: neutral>
<emotion : neutral>
<emotion: neutral>
<emotion: surprise>
<emotion: neutral>
<emotion : neutral>
<emotion: surprise>
<emotion: surprise>
<emotion: neutral>
<emotion: neutral>
<emotion: surprise>
<emotion: neutral>
<emotion: surprise>
<emotion: disgust>


: 

In [ ]:
# Prompt simple + Only Utterance
for index, row in df.iterrows():
    completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": ''' predict the emotion of this utterance among <neutral, anger, surprise, joy, disgust, sadness, fear>. 
     Must predict one emotion among seven emotions.
     your output format should be next. 
     <emotion : {your_prediction}>
'''},
    {"role": "user", "content": f"{row['Utterance']})"}  
  ]
)
    assistant_response = completion.choices[0].message.content
    print(assistant_response)
    df.at[index, 'prediction'] = assistant_response
    df.to_csv(r"C:\Users\Yoon\OneDrive\한양대학교\3-1\인공지능\pj\pj\gandan_GPT_4o_History_final_output.csv")


In [ ]:
# prompt simple + single utterance + PE
for index, row in df3.iterrows():
    completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": '''You are a psychologist with great ability to predict the feelings of the last utterance of the following drama conversation. 

1. The last utterance is speaking with one of the following seven emotions. Never predict another emotion except for these seven sentiments. 
<neutral, anger, surprise, joy, disgust, sadness, fear>
You must predict the last utterspeaker's emotion among seven emotions.

2. Your output format should be next. 
<emotion : {your_prediction}>

You can't talk about the basis of judgment, the situation, only to keep that form all the time.
For example, if you judged that the emotion of the last utterance was a specific sentiment1, your output should be <emotion: {sentiment1}>.'''},
    {"role": "user", "content":  f" here is the utterance. \n{row['Utterance']}"}
  ]
)
    assistant_response = completion.choices[0].message.content
    print(assistant_response)
    df3.at[index, 'prediction'] = assistant_response
    df3.to_csv(r"C:\Users\Yoon\OneDrive\한양대학교\3-1\인공지능\pj\pj\con_onlyUT_GPT_4o_History_final_output.csv")


<emotion: disgust>
<emotion: surprise>
<emotion: neutral>
<emotion: surprise>
<emotion: surprise>
<emotion: surprise>
<emotion: anger>
<emotion: surprise>
<emotion: surprise>
<emotion: surprise>
<emotion: neutral>
<emotion: neutral>
<emotion: surprise>
<emotion: disgust>
<emotion: joy>
<emotion: joy>
<emotion: anger>
<emotion: anger>
<emotion: anger>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: disgust>
<emotion: surprise>
<emotion: neutral>
<emotion: joy>
<emotion: fear>
<emotion: joy>
<emotion: surprise>
<emotion: surprise>
<emotion: sadness>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: neutral>
<emotion: joy>
<emotion: joy>
<emotion: neutral>
<emotion: confusion>
<emotion: anger>
<emotion: anger>
<emotion: anger>
<emotion: anger>
<emotion: anger>
<emotion: neutral>
<emotion: neutral>
<emotion: fear>
<emotion: neutral>
<emotion: neutral>
<emotion: neutral>
<emotion: joy>
<emotion: sadness>
<emotion: anger>
<emotion: neutral>
<emotion: neutral>


In [ ]:
# accuracy
import re

def extract_emotion(prediction):
    match = re.search(r'<emotion: (.*?)>', prediction)
    match2 = re.search(r'<emotion : (.*?)>', prediction)
    if match :
        return match.group(1)
    elif match2 :
        return match2.group(1)
    return None

df['extracted_emotion'] = df['prediction'].apply(extract_emotion)

accuracy = (df['Emotion'] == df['extracted_emotion']).mean()

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 54.83%
